<a href="https://colab.research.google.com/github/Rushi908/Langchain/blob/main/Contextual_Compression_Reteriver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Link of Article**
https://blog.langchain.dev/improving-document-retrieval-with-contextual-compression/

# **This is the problem**

When you store data in a document system, you usually don’t know in advance what questions people will ask to retrieve that data. In the example of a Q&A system, we divided the text into equal chunks, without considering what parts of the text might actually answer a question.

The problem is that when a user asks a specific question, the retrieved chunk might contain some useful information but also unrelated or unnecessary details. Including this irrelevant information in the prompt for the language model (LLM) is a problem because:

1. **It can confuse the LLM**: The extra, irrelevant information might distract the model from focusing on the important details needed to answer the question.
2. **It wastes space**: The prompt has a size limit, so unnecessary information takes up room that could be better used for more relevant content.

This reduces the accuracy and efficiency of the response.

# **Three techniques to solve issue**
# **1.DocumentCompressor**

# **2.LLMChainExtractor**
The LLMChainExtractor uses an LLMChain to extract from each document only the statements that are relevant to the query.

# **3.EmbeddingsFilter**
The EmbeddingsFilter embeds both the retrieved documents and the query and filters out any documents whose embeddings aren’t sufficiently similar to the embedded query. On it’s own this compressor does something very similar to most VectorStore retrievers, but it becomes more useful as a component in…
… the DocumentCompressorPipeline, which makes it easy to create a pipeline of transformations and compressors and run them in sequence. A simple example of this is you may want to combine a TextSplitter and an EmbeddingsFilter to first break up your documents into smaller pieces and then filter out the split documents that are no longer relevant.


# **Download the libaries**

In [2]:
!pip -q install langchain langchain_community  faiss-cpu openai tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.0 MB/s eta 0:00:00


# **load the Data**

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter

In [4]:
def load_text_data(file_path):
  loader=TextLoader(file_path)
  documents=loader.load()
  return documents

In [5]:
document=load_text_data("/content/state_of_the_union.txt")

# **Convert Text into the Chunk**

In [6]:
def split_text(documents):
  text_splitter=CharacterTextSplitter(chunk_size=500,chunk_overlap=100)
  docs=text_splitter.split_documents(documents)
  return docs

In [7]:
split_text=split_text(document)

# **Convert into Embedding**

In [8]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

<ipython-input-8-7b32e592d91b>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
retriever=FAISS.from_documents(split_text,embedding).as_retriever()

In [10]:
docs = retriever.invoke("What did the president say about Ketanji Brown Jackson")

In [11]:
docs

[Document(metadata={'source': '/content/state_of_the_union.txt'}, page_content='One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.'),
 Document(metadata={'source': '/content/state_of_the_union.txt'}, page_content='We cannot let this happen. \n\nTonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service.'),
 Document(met

In [12]:
# Helper function for printing docs

def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [13]:
pretty_print_docs(docs)

Document 1:

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
----------------------------------------------------------------------------------------------------
Document 2:

We cannot let this happen. 

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service.
----------------------------------------------------------

In [14]:
docs2 = retriever.invoke("What were the top three priorities outlined in the most recent State of the Union address?")
docs2

[Document(metadata={'source': '/content/state_of_the_union.txt'}, page_content='Because I see the future that is within our grasp. \n\nBecause I know there is simply nothing beyond our capacity. \n\nWe are the only nation on Earth that has always turned every crisis we have faced into an opportunity. \n\nThe only nation that can be defined by a single word: possibilities. \n\nSo on this night, in our 245th year as a nation, I have come to report on the State of the Union. \n\nAnd my report is this: the State of the Union is strong—because you, the American people, are strong.'),
 Document(metadata={'source': '/content/state_of_the_union.txt'}, page_content='Third – we can end the shutdown of schools and businesses. We have the tools we need. \n\nIt’s time for Americans to get back to work and fill our great downtowns again.  People working from home can feel safe to begin to return to the office.   \n\nWe’re doing that here in the federal government. The vast majority of federal worker

In [15]:
from huggingface_hub import notebook_login
notebook_login()

In [17]:
from langchain_community.llms import HuggingFacePipeline
llm=HuggingFacePipeline.from_model_id(
                                  model_id="meta-llama/Llama-3.2-1B",
                                  task="text-generation",
                                   pipeline_kwargs=dict(
                                       max_new_tokens=512,
                                        do_sample=False,
                                       repetition_penalty=1.03,
                                        return_full_text=False,
                                             ),
                                            device=0
                                          )

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [18]:
from langchain.chains import RetrievalQA
chain = RetrievalQA.from_chain_type(llm=llm,retriever=retriever)

In [19]:
query="What were the top three priorities outlined in the most recent State of the Union address?"

In [20]:
answer=chain.invoke(query)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [21]:
print(answer)

{'query': 'What were the top three priorities outlined in the most recent State of the Union address?', 'result': ' The top three priorities outlined in the most recent State of the Union address were: (1) ending the shutdown of schools and businesses, (2) passing the Paycheck Fairness Act and paid leave, and (3) strengthening the Violence Against Women Act.'}


In [22]:
print(chain.invoke(query)['result'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


 The top three priorities outlined in the most recent State of the Union address were: (1) ending the shutdown of schools and businesses, (2) passing the Paycheck Fairness Act and paid leave, and (3) strengthening the Violence Against Women Act.


In [23]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [24]:
compressor=LLMChainExtractor.from_llm(llm)

In [25]:
compression_retriever=ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)

In [26]:
compressed_docs = compression_retriever.invoke("What did the president say about Ketanji Jackson Brown")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:5

In [27]:
compressed_docs

[Document(metadata={'source': '/content/state_of_the_union.txt'}, page_content='>>>\n> One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\n> And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. \n\n> One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.'),
 Document(metadata={'source': '/content/state_of_the_union.txt'}, page_content='>>>\nWe cannot let this happen. \n\nTonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your servi

In [28]:
compressed_docs = compression_retriever.invoke("What were the top three priorities outlined in the most recent State of the Union address?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:5

In [29]:
compressed_docs

[Document(metadata={'source': '/content/state_of_the_union.txt'}, page_content='>>>\nBecause I see the future that is within our grasp. \n\nBecause I know there is simply nothing beyond our capacity. \n\nWe are the only nation on Earth that has always turned every crisis we have faced into an opportunity. \n\nThe only nation that can be defined by a single word: possibilities. \n\nSo on this night, in our 245th year as a nation, I have come to report on the State of the Union. \n\nAnd my report is this: the State of the Union is strong—because you, the American people, are strong.'),
 Document(metadata={'source': '/content/state_of_the_union.txt'}, page_content=">>>\nThe vast majority of federal workers will once again work in person. \n\nOur schools are open. Let’s keep it that way. Our kids need to be in school.\n\n## Answer\n\nThe vast majority of federal workers will once again work in person. Our schools are open. Let's keep it that way. Our kids need to be in school."),
 Document

In [30]:
pretty_print_docs(compressed_docs)

Document 1:

>>>
Because I see the future that is within our grasp. 

Because I know there is simply nothing beyond our capacity. 

We are the only nation on Earth that has always turned every crisis we have faced into an opportunity. 

The only nation that can be defined by a single word: possibilities. 

So on this night, in our 245th year as a nation, I have come to report on the State of the Union. 

And my report is this: the State of the Union is strong—because you, the American people, are strong.
----------------------------------------------------------------------------------------------------
Document 2:

>>>
The vast majority of federal workers will once again work in person. 

Our schools are open. Let’s keep it that way. Our kids need to be in school.

## Answer

The vast majority of federal workers will once again work in person. Our schools are open. Let's keep it that way. Our kids need to be in school.
------------------------------------------------------------------

In [31]:
compressed_docs2 = compression_retriever.invoke("How did the President propose to tackle the issue of climate change?")

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperatur

In [32]:
compressed_docs2

[Document(metadata={'source': '/content/state_of_the_union.txt'}, page_content='>>>\n> Second – cut energy costs for families an average of $500 a year by combatting climate change.  \n> Let’s provide investments and tax credits to weatherize your homes and businesses to be energy efficient and you get a tax credit; double America’s clean energy production in solar, wind, and so much more;  lower the price of electric vehicles, saving you another $80 a month because you’ll never have to pay at the gas pump again.'),
 Document(metadata={'source': '/content/state_of_the_union.txt'}, page_content='>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>\n>>>

In [33]:
pretty_print_docs(compressed_docs2)

Document 1:

>>>
> Second – cut energy costs for families an average of $500 a year by combatting climate change.  
> Let’s provide investments and tax credits to weatherize your homes and businesses to be energy efficient and you get a tax credit; double America’s clean energy production in solar, wind, and so much more;  lower the price of electric vehicles, saving you another $80 a month because you’ll never have to pay at the gas pump again.
----------------------------------------------------------------------------------------------------
Document 2:

>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>
>>>


In [34]:
from langchain.retrievers.document_compressors import LLMChainFilter
filter=LLMChainFilter.from_llm(llm)

In [35]:
compression_retriever2 = ContextualCompressionRetriever(base_compressor=filter, base_retriever=retriever)

In [36]:
compressed_docs3 = compression_retriever2.invoke("What were the top three priorities outlined in the most recent State of the Union address?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


ValueError: Ambiguous response. Both YES and NO in received:  
>>>
> Relevant (YES / NO): 
>>>
> Relevant (YES / NO): 
>>>
> Relevant (YES / NO): 
>>>
> Relevant (YES / NO): 
>>>
> Relevant (YES / NO): 
>>>
> Relevant (YES / NO): 
>>>
> Relevant (YES / NO): 
>>>
> Relevant (YES / NO): 
>>>
> Relevant (YES / NO): 
>>>
> Relevant (YES / NO): 
>>>
> Relevant (YES / NO): 
>>>
> Relevant (YES / NO): 
>>>
> Relevant (YES / NO): 
>>>
> Relevant (YES / NO): 
>>>
> Relevant (YES / NO): 
>>>
> Relevant (YES / NO): 
>>>
> Relevant (YES / NO): 
>>>
> Relevant (YES / NO): 
>>>
> Relevant (YES / NO): 
>>>
> Relevant (YES / NO): 
>>>
> Relevant (YES / NO): 
>>>
> Relevant (YES / NO): 
>>>
> Relevant (YES / NO): 
>>>
> Relevant (YES / NO): 
>>>
> Relevant (YES / NO): 
>>>
> Relevant (YES / NO): 
>>>
> Relevant (YES / NO): 
>>>
> Relevant (YES / NO): 
>>>
> Relevant (YES / NO): 
>>>
> Relevant (YES / NO): 
>>>
> Relevant (YES / NO): 
>>>
> Relevant (YES / NO): 
>>>
> Relevant (YES / NO): 
>>>
> Relevan

In [37]:
compressed_docs3

NameError: name 'compressed_docs3' is not defined

In [38]:
original_contexts_len = len("\n\n".join([d.page_content for i, d in enumerate(docs2)]))
original_contexts_len

1817

In [39]:
compressed_contexts_len = len("\n\n".join([d.page_content for i, d in enumerate(compressed_docs)]))

In [40]:
compressed_contexts_len

1458

In [41]:
print("Original context length:", original_contexts_len)

Original context length: 1817


In [42]:
print("Compressed context length:", compressed_contexts_len)

Compressed context length: 1458


In [43]:
print("Compressed Ratio:", f"{original_contexts_len/(compressed_contexts_len + 1e-5):.2f}x")

Compressed Ratio: 1.25x


In [52]:
from langchain.retrievers.document_compressors import EmbeddingsFilter

In [53]:
embeddings_filter = EmbeddingsFilter(embeddings=embedding)

In [55]:
compression_retriever3 = ContextualCompressionRetriever(base_compressor=embeddings_filter, base_retriever=retriever)

In [56]:
compressed_docs4 = compression_retriever3.invoke("What were the top three priorities outlined in the most recent State of the Union address?")

In [57]:
pretty_print_docs(compressed_docs4)

Document 1:

Because I see the future that is within our grasp. 

Because I know there is simply nothing beyond our capacity. 

We are the only nation on Earth that has always turned every crisis we have faced into an opportunity. 

The only nation that can be defined by a single word: possibilities. 

So on this night, in our 245th year as a nation, I have come to report on the State of the Union. 

And my report is this: the State of the Union is strong—because you, the American people, are strong.
----------------------------------------------------------------------------------------------------
Document 2:

Third – we can end the shutdown of schools and businesses. We have the tools we need. 

It’s time for Americans to get back to work and fill our great downtowns again.  People working from home can feel safe to begin to return to the office.   

We’re doing that here in the federal government. The vast majority of federal workers will once again work in person. 

Our schools ar

In [58]:
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain_text_splitters import CharacterTextSplitter

In [59]:
splitter=CharacterTextSplitter(chunk_size=500,chunk_overlap=0,separator=".")

In [60]:
redundant_filter = EmbeddingsRedundantFilter(embeddings=embedding)

In [61]:
redundant_filter

EmbeddingsRedundantFilter(embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), similarity_fn=<function cosine_similarity at 0x7a67ff5f35b0>, similarity_threshold=0.95)

In [62]:
relevant_filter = EmbeddingsFilter(embeddings=embedding, similarity_threshold=0.76)

In [63]:
pipeline_compressor = DocumentCompressorPipeline(transformers=[splitter, redundant_filter, relevant_filter])

In [64]:
compression_retriever = ContextualCompressionRetriever(base_compressor=pipeline_compressor, base_retriever=retriever)

In [65]:
compressed_docs = compression_retriever.invoke("What were the top three priorities outlined in the most recent State of the Union address?")

In [66]:
pretty_print_docs(compressed_docs)

In [67]:
from langchain.chains import RetrievalQA

In [68]:
chain = RetrievalQA.from_chain_type(llm=llm, retriever=compression_retriever)

In [69]:
query="What were the top three priorities outlined in the most recent State of the Union address?"

In [70]:
chain.invoke(query)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


{'query': 'What were the top three priorities outlined in the most recent State of the Union address?',
 'result': ' The top three priorities outlined in the most recent State of the Union address were: (1) the economy, (2) national security, and (3) education.'}

In [71]:
print(chain.invoke(query)['result'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


 The top three priorities outlined in the most recent State of the Union address were: (1) the economy, (2) national security, and (3) education.
